In [1]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf

2023-06-01 12:00:44.166009: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dev = qml.device('default.mixed', wires=3)

In [3]:
def real():
    qml.Rot( 0.0,np.pi/2, np.pi/6,wires=0)

In [5]:
def generator(w,**kwargs):
    qml.RX(w[0], wires=1)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=1)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=1)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[1,2])
    qml.RX(w[6], wires=1)
    qml.RY(w[7], wires=1)
    qml.RZ(w[8], wires=1)
    qml.Rot(0.0, np.pi/2, -np.pi/6,wires=1)
    qml.BitFlip(p, wires=1)
    #qml.BitFlip(p, wires=2)

In [6]:
def discriminator(w):
    qml.Hadamard(wires=0)
    qml.CZ(wires=[1, 0])
    qml.RZ(w[0], wires=0)
    qml.RZ(w[1], wires=1)
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.BitFlip(p, wires=0)
    qml.BitFlip(p, wires=1)

In [7]:
@qml.qnode(dev, interface='tf')
def real_gen_disc_circuit(gen_weights, disc_weights):
    real()
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval((qml.PauliZ(0)+qml.Identity(0))@ (qml.PauliZ(1)+qml.Identity(1)))

In [8]:
def prob_discriminated(gen_weights,disc_weights):
    discriminated_output = real_gen_disc_circuit(gen_weights, disc_weights)
    #convert to probability
    prob_discriminated_output = (discriminated_output)/4
    return prob_discriminated_output